In [ ]:
import os
import uuid
import boto3
from datetime import datetime
from IPython.display import display, Image

# AWS setup
BUCKET_NAME = "your-s3-bucket-name"
TABLE_NAME = "your-dynamodb-table-name"

# Initialize AWS clients
s3_client = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(TABLE_NAME)

print("AWS services initialized.")

In [ ]:
def upload_image(file_path, category=None):
    file_id = str(uuid.uuid4())
    file_extension = os.path.splitext(file_path)[1].lower()
    s3_key = f"memes/{file_id}{file_extension}"
    
    # Upload to S3
    s3_client.upload_file(file_path, BUCKET_NAME, s3_key)
    
    # Add to DynamoDB
    table.put_item(
        Item={
            'id': file_id,
            's3_key': s3_key,
            'original_filename': os.path.basename(file_path),
            'category': category,
            'upload_date': datetime.now().isoformat()
        }
    )
    
    print(f"Uploaded {file_path} to S3 with key: {s3_key}")
    print(f"Added entry to DynamoDB with ID: {file_id}")
    
    return file_id

print("upload_image function defined.")

In [ ]:
def list_local_images(directory):
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif')
    return [f for f in os.listdir(directory) if f.lower().endswith(image_extensions)]

def display_image(file_path):
    display(Image(filename=file_path, width=300))

print("Utility functions defined.")

In [ ]:
# Example usage for a single file
image_directory = "path/to/your/image/directory"
image_files = list_local_images(image_directory)

if image_files:
    file_to_upload = os.path.join(image_directory, image_files[0])
    print(f"Uploading: {file_to_upload}")
    display_image(file_to_upload)
    
    file_id = upload_image(file_to_upload, category="example")
    print(f"Uploaded with ID: {file_id}")
else:
    print("No image files found in the specified directory.")

In [ ]:
# Example usage for multiple files
image_directory = "path/to/your/image/directory"
image_files = list_local_images(image_directory)

for image_file in image_files[:5]:  # Limit to first 5 images for this example
    file_to_upload = os.path.join(image_directory, image_file)
    print(f"\nUploading: {file_to_upload}")
    display_image(file_to_upload)
    
    file_id = upload_image(file_to_upload, category="batch_upload")
    print(f"Uploaded with ID: {file_id}")

print("\nBatch upload complete.")

In [ ]:
def verify_upload(file_id):
    try:
        response = table.get_item(Key={'id': file_id})
        item = response.get('Item')
        if item:
            print(f"File with ID {file_id} found in DynamoDB.")
            s3_key = item['s3_key']
            s3_client.head_object(Bucket=BUCKET_NAME, Key=s3_key)
            print(f"File with key {s3_key} found in S3 bucket.")
            return True
        else:
            print(f"File with ID {file_id} not found in DynamoDB.")
            return False
    except Exception as e:
        print(f"Error verifying upload: {str(e)}")
        return False

# Example usage
file_id_to_verify = "paste_a_file_id_here"
verify_upload(file_id_to_verify)